<a href="https://colab.research.google.com/github/GerardoG2/titanic-project/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>